<a href="https://colab.research.google.com/github/tyoc213/fastai_xla_extensions/blob/master/explore_nbs/Basic_lenet_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install fastai2 from github

In [ ]:
!pip uninstall -y fastai fastai2 fastcore

In [ ]:
VERSION = "20200707"  #"20200515" @param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py

In [ ]:
#!TORCH_SHOW_CPP_STACKTRACES=1 python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev
!python pytorch-xla-env-setup.py  --version $VERSION --apt-packages libomp5 libopenblas-dev

In [ ]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm

In [ ]:
!pip install git+https://github.com/fastai/fastcore
!pip install git+https://github.com/fastai/fastai

In [ ]:
from fastai.vision.all import *

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)
path.ls()[2].ls()

In [ ]:
def get_my_labels(fname):
    return int(fname.parent.name[0])

dblock = DataBlock(
    splitter = RandomSplitter(),
    #item_tfms = Resize(128),
    blocks = (ImageBlock, CategoryBlock),
    get_items = get_image_files,
    get_y = get_my_labels
)


In [ ]:
dls_normal = dblock.dataloaders(path)
dls_normal.vocab

# Lenet with convs and F.max_pool2d

In [ ]:
class MyLenet(nn.Module):
    def __init__(self):
        super(MyLenet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.conv2 = nn.Conv2d(6,16,3)
        self.hiden4 = nn.Linear(400, 2) # 2 outputs instead of 10
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = self.hiden4(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

lenet = MyLenet()
learn = Learner(dls_normal, lenet, metrics=[error_rate, accuracy])



In [ ]:
%%time
learn.fit_one_cycle(1)
learn.validate()

# Lenet with layers

In [ ]:
class Lenet2(nn.Module):
    def __init__(self):
        super(Lenet2, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2) # Only 2 outputs instead of 10
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features



lenet2 = Lenet2()
learn2 = Learner(dls_normal, lenet2, metrics=[error_rate, accuracy])



In [ ]:
%%time
learn2.fit_one_cycle(1)
learn.validate()

# Current TPU implementation callbacks

Just import fastai_xla_extensions

In [ ]:
import fastai_xla_extensions.core

# Load in TPU

In [ ]:
dblock.summary(path)

# Use TPU device

In [ ]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm


# Aquire a TPU device

In [ ]:

dede = xm.xla_device()
dls_tpu = dblock.dataloaders(path, device=dede)
dls_tpu.show_batch(), dls_tpu.vocab

In [ ]:
dls_tpu.show([1])

In [ ]:
#sgd_xla_opt = XLAOptFuncWrapper(SGD)
#adam_xla_opt = XLAOptFuncWrapper(Adam)


## First training on TPU

In [ ]:
# from fastai.callback.all import *
from fastai.test_utils import *
lenet_tpu = Lenet2()
tpu_learner = Learner(dls_tpu,
                      lenet_tpu,
                      device=dede,
                      metrics=accuracy, 
                      loss_func=F.cross_entropy,
                      cbs=[XLAOptCallback()])

In [ ]:
tpu_learner.summary()

In [ ]:
tpu_learner.show_training_loop()

In [ ]:
dls_tpu.device

# Call fit

Will fail in `self.loss.backward(); `?

In [ ]:
tpu_learner.fit(1)

In [ ]:
tpu_learner.lr_find()

reference for https://pytorch.org/docs/stable/autograd.html#torch.autograd.backward

In [ ]:
ob = tpu_learner.dls.train.one_batch()
#print(ob)
len(ob), ob[0].shape, ob[1].shape, F.cross_entropy, tpu_learner.pred

In [ ]:
#the_loss = tpu_learner.loss_func(tpu_learner.pred, *tpu_learner.yb)
the_loss = tpu_learner.loss_func(tpu_learner.dls, *tpu_learner.yb)